### Import dependencies

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### import data

In [4]:
df = pd.read_csv("medical_examination.csv")

df.head()

,id,age,sex,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


### Add an overweight column to the data.
To determine if a person is overweight, first calculate their BMI by dividing their weight in kilograms by the square of their height in meters. If that value is > 25 then the person is overweight. Use the value 0 for NOT overweight and the value 1 for overweight.

In [33]:
bmi = df["weight"] / (df["height"] / 100) ** 2
df["overweight"] = (bmi > 25).astype(int)

### Normalize the data by making 0 always good and 1 always bad. 
If the value of cholesterol or gluc is 1, make the value 0. If the value is more than 1, make the value 1.

In [38]:
df[["cholesterol", "gluc"]] = (df[["cholesterol", "gluc"]] > 1).astype(int)

In [41]:
df.sample(n=5, frac=0.1)

ValueError: Please enter a value for `frac` OR `n`, not both